In [15]:
import os
import pandas as pd

# Шлях до папки з CSV-файлами
input_dir = "filtered_data"
# Шлях до вихідного файлу
output_file = "weather.csv"

# Перевірка існування папки
if not os.path.exists(input_dir):
    print(f"Помилка: Папка {input_dir} не існує")
    exit(1)

# Список для зберігання унікальних значень test_date
unique_dates = set()

# Проходимо по всіх файлах у папці
for filename in os.listdir(input_dir):
    if filename.endswith(".csv"):
        file_path = os.path.join(input_dir, filename)
        try:
            # Читаємо CSV-файл
            df = pd.read_csv(file_path)
            # Перевіряємо наявність колонки test_date
            if "test_date" in df.columns:
                # Витягуємо унікальні значення test_date і додаємо до множини
                unique_dates.update(df["test_date"].dropna().unique())
            else:
                print(f"Попередження: У файлі {filename} відсутня колонка test_date")
        except Exception as e:
            print(f"Помилка при обробці файлу {filename}: {str(e)}")

# Якщо не знайдено жодних дат
if not unique_dates:
    print("Помилка: Не знайдено унікальних значень у колонці test_date")
    exit(1)

# Створюємо DataFrame з унікальними датами
result_df = pd.DataFrame(sorted(unique_dates), columns=["test_date"])

# Зберігаємо результат у CSV-файл
try:
    result_df.to_csv(output_file, index=False)
    print(f"Успіх: Унікальні значення test_date збережено у {output_file}")
    print(f"Кількість унікальних дат: {len(unique_dates)}")
except Exception as e:
    print(f"Помилка при збереженні файлу {output_file}: {str(e)}")

/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/206837318.py:23: DtypeWarning: Columns (96,97,100,102,103,104,105,116,117,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Попередження: У файлі 2020.csv відсутня колонка test_date


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/206837318.py:23: DtypeWarning: Columns (117,118,121,123,124,125,126,137,138,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Попередження: У файлі 2021.csv відсутня колонка test_date


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/206837318.py:23: DtypeWarning: Columns (8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/206837318.py:23: DtypeWarning: Columns (100,116,117,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Попередження: У файлі 2019.csv відсутня колонка test_date
Успіх: Унікальні значення test_date збережено у weather.csv
Кількість унікальних дат: 46


In [16]:
import pandas as pd
import requests
import os
from datetime import datetime

# Шлях до файлу з датами
input_file = "weather.csv"
# Шлях до вихідного файлу з погодними даними
output_file = "weather_data.csv"

# Координати для Києва (зміни на потрібне місто в Україні)
city = "Kyiv"
latitude = 50.4501  # Широта Києва
longitude = 30.5234  # Довгота Києва

# Словник із координатами інших міст України (для зручності)
ukraine_cities = {
    "Kyiv": (50.4501, 30.5234),
    "Lviv": (49.8397, 24.0297),
    "Odesa": (46.4825, 30.7233),
    "Kharkiv": (49.9935, 36.2304),
    "Dnipro": (48.4647, 35.0462),
    "Zaporizhzhia": (47.8388, 35.1396)
}

# Вибір координат для міста
if city in ukraine_cities:
    latitude, longitude = ukraine_cities[city]
else:
    print(f"Помилка: Місто {city} не знайдено. Використовуйте одне з: {list(ukraine_cities.keys())}")
    exit(1)

# Перевірка існування файлу weather.csv
if not os.path.exists(input_file):
    print(f"Помилка: Файл {input_file} не існує")
    exit(1)

# Читання дат із weather.csv
try:
    df_dates = pd.read_csv(input_file)
    if "test_date" not in df_dates.columns:
        print(f"Помилка: Колонка test_date відсутня у файлі {input_file}")
        exit(1)
    dates = df_dates["test_date"].unique()
except Exception as e:
    print(f"Помилка при читанні файлу {input_file}: {str(e)}")
    exit(1)

# Список для зберігання погодних даних
weather_data = []

# Базовий URL для Open-Meteo Historical Weather API
base_url = "https://archive-api.open-meteo.com/v1/archive"

# Проходимо по кожній даті
for date in dates:
    try:
        # Перевірка формату дати
        datetime.strptime(date, "%Y-%m-%d")
        # Параметри запиту
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "start_date": date,
            "end_date": date,
            "hourly": "temperature_2m,precipitation,cloudcover,wind_speed_10m,relative_humidity_2m",
            "timezone": "Europe/Kiev"
        }
        # Надсилання запиту до API
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Перевірка на помилки HTTP
        data = response.json()

        # Витягуємо погодні дані
        if "hourly" in data:
            hourly_data = data["hourly"]
            for time, temp, precip, cloud, wind, humidity in zip(
                hourly_data["time"],
                hourly_data["temperature_2m"],
                hourly_data["precipitation"],
                hourly_data["cloudcover"],
                hourly_data["wind_speed_10m"],
                hourly_data["relative_humidity_2m"]
            ):
                # Час у локальному форматі
                time_local = datetime.strptime(time, "%Y-%m-%dT%H:%M")
                # Опис погоди на основі опадів і хмарності
                weather_desc = "Sunny" if cloud < 20 and precip == 0 else \
                              "Cloudy" if cloud < 80 and precip == 0 else \
                              "Rainy" if precip > 0 else "Overcast"
                weather_data.append({
                    "city": city,
                    "date": date,
                    "time": time_local.strftime("%H:%M"),
                    "temperature_c": temp,
                    "precipitation_mm": precip,
                    "cloudcover_percent": cloud,
                    "wind_speed_kmh": wind,
                    "relative_humidity_percent": humidity,
                    "weather_description": weather_desc
                })
        else:
            print(f"Попередження: Дані для дати {date} у місті {city} відсутні")
    except ValueError:
        print(f"Попередження: Некоректний формат дати {date}, пропускаємо")
    except requests.RequestException as e:
        print(f"Помилка при запиті для дати {date}: {str(e)}")
    except Exception as e:
        print(f"Інша помилка для дати {date}: {str(e)}")

# Створюємо DataFrame із погодними даними
if weather_data:
    result_df = pd.DataFrame(weather_data)
    # Зберігаємо у CSV
    try:
        result_df.to_csv(output_file, index=False)
        print(f"Успіх: Погодні дані для {city} збережено у {output_file}")
        print(f"Кількість записів: {len(result_df)}")
    except Exception as e:
        print(f"Помилка при збереженні файлу {output_file}: {str(e)}")
else:
    print(f"Помилка: Не вдалося отримати погодних даних для {city}")

Помилка при запиті для дати 2024-06-07: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Успіх: Погодні дані для Kyiv збережено у weather_data.csv
Кількість записів: 1080


In [17]:
import pandas as pd
import os

def get_daily_weather_id(input_file="weather_data.csv", output_file="daily_weather.csv"):
    """
    Функція витягує загальний ідентифікатор погоди за день із погодинних даних.
    
    Параметри:
        input_file (str): Шлях до вхідного файлу (weather_data.csv).
        output_file (str): Шлях до вихідного файлу (daily_weather.csv).
    
    Повертає:
        None (зберігає результат у CSV).
    """
    # Перевірка існування файлу
    if not os.path.exists(input_file):
        print(f"Помилка: Файл {input_file} не існує")
        return

    # Читання даних
    try:
        df = pd.read_csv(input_file)
        required_columns = ["city", "date", "weather_description"]
        if not all(col in df.columns for col in required_columns):
            print(f"Помилка: У файлі {input_file} відсутні необхідні колонки: {required_columns}")
            return
    except Exception as e:
        print(f"Помилка при читанні файлу {input_file}: {str(e)}")
        return

    # Групування за містом і датою, вибір найпоширенішого weather_description
    try:
        daily_weather = df.groupby(["city", "date"])["weather_description"].agg(lambda x: x.mode()[0]).reset_index()
        daily_weather = daily_weather.rename(columns={"weather_description": "weather_id"})

        # Збереження результату
        daily_weather.to_csv(output_file, index=False)
        print(f"Успіх: Загальні ідентифікатори погоди збережено у {output_file}")
        print(f"Кількість записів: {len(daily_weather)}")
    except Exception as e:
        print(f"Помилка при обробці або збереженні даних: {str(e)}")

# Виклик функції
if __name__ == "__main__":
    get_daily_weather_id()

Успіх: Загальні ідентифікатори погоди збережено у daily_weather.csv
Кількість записів: 45


In [18]:
import pandas as pd
import os

def add_weather_id_to_transformed_data(daily_weather_file="daily_weather.csv", input_dir="filtered_data"):
    """
    Додає стовпець weather_id до всіх CSV-файлів у папці transformed_data на основі дат із daily_weather.csv.
    
    Параметри:
        daily_weather_file (str): Шлях до файлу з ідентифікаторами погоди (daily_weather.csv).
        input_dir (str): Папка з CSV-файлами (transformed_data).
    
    Повертає:
        None (оновлює файли в input_dir).
    """
    # Перевірка існування daily_weather.csv
    if not os.path.exists(daily_weather_file):
        print(f"Помилка: Файл {daily_weather_file} не існує")
        return

    # Читання daily_weather.csv
    try:
        df_weather = pd.read_csv(daily_weather_file)
        required_columns = ["date", "weather_id"]
        if not all(col in df_weather.columns for col in required_columns):
            print(f"Помилка: У файлі {daily_weather_file} відсутні необхідні колонки: {required_columns}")
            return
        # Створюємо словник {date: weather_id}
        weather_dict = dict(zip(df_weather["date"], df_weather["weather_id"]))
    except Exception as e:
        print(f"Помилка при читанні файлу {daily_weather_file}: {str(e)}")
        return

    # Перевірка існування папки transformed_data
    if not os.path.exists(input_dir):
        print(f"Помилка: Папка {input_dir} не існує")
        return

    # Лічильник оброблених файлів
    processed_files = 0

    # Проходимо по всіх CSV-файлах у папці
    for filename in os.listdir(input_dir):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_dir, filename)
            try:
                # Читаємо CSV-файл
                df = pd.read_csv(file_path)
                # Перевіряємо наявність колонки test_date
                if "test_date" not in df.columns:
                    print(f"Попередження: У файлі {filename} відсутня колонка test_date, пропускаємо")
                    continue

                # Додаємо стовпець weather_id
                df["weather_id"] = df["test_date"].map(weather_dict)

                # Перевіряємо, чи всі дати мають weather_id
                if df["weather_id"].isna().any():
                    missing_dates = df[df["weather_id"].isna()]["test_date"].unique()
                    print(f"Попередження: У файлі {filename} не знайдено weather_id для дат: {missing_dates}")

                # Зберігаємо оновлений файл
                df.to_csv(file_path, index=False)
                processed_files += 1
                print(f"Оброблено файл: {filename}")
            except Exception as e:
                print(f"Помилка при обробці файлу {filename}: {str(e)}")

    if processed_files == 0:
        print("Помилка: Жоден CSV-файл не було оброблено")
    else:
        print(f"Успіх: Оброблено {processed_files} файлів у папці {input_dir}")

# Виклик функції
if __name__ == "__main__":
    add_weather_id_to_transformed_data()

/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/4238914256.py:47: DtypeWarning: Columns (96,97,100,102,103,104,105,116,117,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Попередження: У файлі 2020.csv відсутня колонка test_date, пропускаємо


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/4238914256.py:47: DtypeWarning: Columns (117,118,121,123,124,125,126,137,138,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Попередження: У файлі 2021.csv відсутня колонка test_date, пропускаємо
Оброблено файл: 2023.csv


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/4238914256.py:47: DtypeWarning: Columns (8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Оброблено файл: 2022.csv


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/4238914256.py:47: DtypeWarning: Columns (100,116,117,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Попередження: У файлі 2019.csv відсутня колонка test_date, пропускаємо
Попередження: У файлі 2024.csv не знайдено weather_id для дат: ['2024-06-07']
Оброблено файл: 2024.csv
Успіх: Оброблено 3 файлів у папці filtered_data


In [19]:
df = pd.read_csv("transformed_data/2021.csv")
df

/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/1078033183.py:1: DtypeWarning: Columns (117,118,121,123,124,125,126,137,138,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("transformed_data/2021.csv")


,id,birth_year,gender,region_name,area_name,territory_name,region_type,territory_type,class_profile,class_language,...,spanish_score,spanish_pt_name,spanish_pt_region,spanish_pt_area,spanish_pt_territory,student_age,region_flag,subjects_count,total_score,average_score
0,8a2abef7-625a-4253-8c14-000fe8a856e5,2003,чоловіча,Дніпропетровська область,м.Кам'янське,Дніпровський район міста,Студент закладу вищої освіти,місто,Молодший спеціаліст,українська,...,NaN,NaN,NaN,NaN,NaN,18,east,4,456.0,114.000000
1,3e975d01-9cc9-4072-bbcd-005b9da75096,2003,чоловіча,Кіровоградська область,м.Кропивницький,Фортечний район міста,Випускник загальноосвітнього навчального закла...,місто,Технологічний,українська,...,NaN,NaN,NaN,NaN,NaN,18,central,4,416.0,104.000000
2,58c926c3-8718-4845-918b-00155b917934,2003,чоловіча,Дніпропетровська область,м.Дніпро,Соборний район міста,Студент закладу вищої освіти,місто,Молодший спеціаліст,українська,...,NaN,NaN,NaN,NaN,NaN,18,east,0,0.0,NaN
3,443f89c2-84e7-4a56-bea3-005c776e0121,2004,жіноча,Кіровоградська область,м.Кропивницький,Фортечний район міста,Випускник загальноосвітнього навчального закла...,місто,Біолого-хімічний,українська,...,NaN,NaN,NaN,NaN,NaN,17,central,4,614.0,153.500000
4,0b7b2e81-a906-412f-ad74-005025fd9d29,2003,жіноча,Івано-Франківська область,Долинський район,м.Долина,Випускник загальноосвітнього навчального закла...,місто,Технологічний,українська,...,NaN,NaN,NaN,NaN,NaN,18,west,4,707.0,176.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389318,e62710ee-74fa-4085-b066-f38fc37ceae7,2003,жіноча,Рівненська область,Демидівський район,смт Демидівка,Учень (слухач) закладу професійної (професійно...,селище міського типу,Кваліфікований робітник,українська,...,NaN,NaN,NaN,NaN,NaN,18,north,0,0.0,NaN
389319,b2e4de44-7ebd-4163-8952-f988a9de1f67,1989,жіноча,Харківська область,Кегичівський район,смт Кегичівка,Учень (слухач) закладу професійної (професійно...,селище міського типу,Кваліфікований робітник,українська,...,NaN,NaN,NaN,NaN,NaN,32,east,0,0.0,NaN
389320,e007c190-1fe1-4658-9eb0-f7cf489fc015,2004,жіноча,Одеська область,Одеська область,м.Білгород-Дністровський,Студент закладу вищої освіти,місто,Молодший спеціаліст,українська,...,NaN,NaN,NaN,NaN,NaN,17,south,0,0.0,NaN
389321,2668b9f3-ddf9-4062-acd7-fdf6e2ca8cc8,2004,чоловіча,Вінницька область,Гайсинський район,с.Чечелівка,Випускник загальноосвітнього навчального закла...,"селище, село",Української філології,українська,...,NaN,NaN,NaN,NaN,NaN,17,central,3,508.0,169.333333


## In Kyiv

In [26]:
import pandas as pd
import os

def group_by_year_weather_and_score(input_dir="filtered_data", output_dir="aggregated_data/weather", output_file="weather_score_summary.csv"):
    """
    Групує дані з CSV-файлів у папці transformed_data за роком і weather_id для м. Київ,
    обчислюючи середній average_score та кількість учнів. Зберігає результат у вказаній папці aggregated_data/weather.
    
    Параметри:
        input_dir (str): Папка з CSV-файлами (transformed_data).
        output_dir (str): Папка для збереження результату (aggregated_data/weather).
        output_file (str): Ім'я вихідного файлу (weather_score_summary.csv).
    
    Повертає:
        None (зберігає результат у CSV).
    """
    # Перевірка існування папки transformed_data
    if not os.path.exists(input_dir):
        print(f"Помилка: Папка {input_dir} не існує")
        return

    # Створення вихідної папки, якщо вона не існує
    os.makedirs(output_dir, exist_ok=True)

    # Повний шлях до вихідного файлу
    output_path = os.path.join(output_dir, output_file)

    # Список для зберігання даних із усіх файлів
    all_data = []

    # Проходимо по всіх CSV-файлах у папці
    for filename in os.listdir(input_dir):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_dir, filename)
            try:
                # Читаємо CSV-файл
                df = pd.read_csv(file_path)
                # Перевіряємо наявність потрібних колонок
                required_columns = ["test_date", "weather_id", "average_score", "pt_region_name"]
                if not all(col in df.columns for col in required_columns):
                    print(f"Попередження: У файлі {filename} відсутні колонки {required_columns}, пропускаємо")
                    continue
                # Фільтруємо лише м. Київ і створюємо копію
                df_kyiv = df[df["pt_region_name"] == "м.Київ"].copy()
                if df_kyiv.empty:
                    print(f"Попередження: У файлі {filename} немає даних для м. Київ")
                    continue
                # Витягуємо рік із test_date
                df_kyiv["year"] = df_kyiv["test_date"].str[:4]
                # Вибираємо потрібні колонки
                all_data.append(df_kyiv[["year", "weather_id", "average_score"]])
            except Exception as e:
                print(f"Помилка при обробці файлу {filename}: {str(e)}")

    # Якщо жоден файл не оброблено
    if not all_data:
        print("Помилка: Жоден CSV-файл не містить потрібних даних для м. Київ")
        return

    # Об’єднуємо всі дані в один DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)

    # Групуємо за year і weather_id, обчислюємо середній average_score та кількість учнів
    try:
        # Спочатку групуємо і обчислюємо середнє
        mean_df = combined_df.groupby(["year", "weather_id"])["average_score"].mean().reset_index()
        mean_df = mean_df.rename(columns={"average_score": "mean_average_score"})
        
        # Додаємо кількість учнів (кількість рядків у групі)
        count_df = combined_df.groupby(["year", "weather_id"]).size().reset_index(name="student_count")
        
        # Об’єднуємо таблиці
        summary_df = pd.merge(mean_df, count_df, on=["year", "weather_id"])
        
        # Округляємо середній бал до 2 знаків
        summary_df["mean_average_score"] = summary_df["mean_average_score"].round(2)

        # Сортуємо за роком і weather_id для зручності
        summary_df = summary_df.sort_values(["year", "weather_id"])

        # Перевіряємо, що є саме 4 стовпчики
        if len(summary_df.columns) != 4:
            print(f"Попередження: Очікувалося 4 стовпчики, але є {len(summary_df.columns)}. Колонки: {summary_df.columns.tolist()}")

        # Зберігаємо результат
        summary_df.to_csv(output_path, index=False)
        print(f"Успіх: Результати групування для м. Київ збережено у {output_path}")
        print(summary_df.to_string(index=False))
    except Exception as e:
        print(f"Помилка при групуванні або збереженні даних: {str(e)}")

# Виклик функції
if __name__ == "__main__":
    group_by_year_weather_and_score()

/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/496319249.py:37: DtypeWarning: Columns (96,97,100,102,103,104,105,116,117,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Попередження: У файлі 2020.csv відсутні колонки ['test_date', 'weather_id', 'average_score', 'pt_region_name'], пропускаємо


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/496319249.py:37: DtypeWarning: Columns (117,118,121,123,124,125,126,137,138,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Попередження: У файлі 2021.csv відсутні колонки ['test_date', 'weather_id', 'average_score', 'pt_region_name'], пропускаємо


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/496319249.py:37: DtypeWarning: Columns (8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_44794/496319249.py:37: DtypeWarning: Columns (100,116,117,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Попередження: У файлі 2019.csv відсутні колонки ['test_date', 'weather_id', 'average_score', 'pt_region_name'], пропускаємо
Успіх: Результати групування для м. Київ збережено у aggregated_data/weather/weather_score_summary.csv
year weather_id  mean_average_score  student_count
2022     Cloudy              154.04           3770
2022   Overcast              153.80           4365
2022      Rainy              154.19           4215
2022      Sunny              154.88           5490
2023     Cloudy              146.30           1703
2023   Overcast              148.19          15775
2023      Sunny              145.92           4623
2024     Cloudy              141.15           2226
2024   Overcast              143.53           6411
2024      Rainy              144.93           6190
2024      Sunny              145.09           7927
